<a href="https://colab.research.google.com/github/msaid1976/Tensor_Flow_Alladdin_Persson/blob/main/3%20-%20Neural%20Networks%20with%20Sequential%20and%20Functional%20API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [12]:
import tensorflow as tf
from tensorflow import keras  # we will use sequential API of Keras and also functional API
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

**Keras difinition:**
Since tensor flow 2.0, Keras has integrated with Tensorflow and it is the official higher level API and it is the GO-TO when we're building nural networksand models in TF

**MNist**
It is dataset for images of digits between 0 and 9. and we have 60 000 training set images and test set of 10 000 Images.

All images ae ingay scale and have only one channel and the pixels pixels are 28X28 so it ios relatively small

In [ ]:
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
print(x_train.shape)

(60000, 28, 28)


In [4]:
print(y_train.shape)

(60000,)


In [14]:
# -1 here means (Flatten the Images), keep whatever the value is on that dimension here is 6000
# .astype("float32") to minimize some of the computation
# instead of having values between 0 and 255 so we devide by 255 to normalize values and make it between 0 and 1

x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

In [ ]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)


In [ ]:
# x_train = tf.convert_to_tensor(x_train)

# Sequential API (Very convenient, not very flexible)
Allow you to have one Input that is mapped to one output and this is the limitation. but if you have case of one input to one output then sequential API is the best one to use

In [15]:
model = keras.Sequential(
    [
        keras.Input(shape=(28 * 28)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(10),
    ]
)

In [16]:
"""
we use from_logits=true when we didn't mention Softmax as activation so
    # automatically send to softmax first then it will map it to sparsecategoricalcrossentropy
"""
# if you remove Sparse and just use CategoricalCrossentropy then we will need to use one hot encoding
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"], # keras will keep track during the training for what is the running accurqacy so far
)

In [17]:
# verbose it prints after each epoch, if removed then it will show a progress bar
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)

Epoch 1/5
1875/1875 - 15s - loss: 0.1866 - accuracy: 0.9433 - 15s/epoch - 8ms/step
Epoch 2/5
1875/1875 - 14s - loss: 0.0801 - accuracy: 0.9747 - 14s/epoch - 8ms/step
Epoch 3/5
1875/1875 - 14s - loss: 0.0546 - accuracy: 0.9833 - 14s/epoch - 7ms/step
Epoch 4/5
1875/1875 - 14s - loss: 0.0405 - accuracy: 0.9866 - 14s/epoch - 7ms/step
Epoch 5/5
1875/1875 - 14s - loss: 0.0326 - accuracy: 0.9893 - 14s/epoch - 7ms/step


In [18]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 1s - loss: 0.0706 - accuracy: 0.9803 - 901ms/epoch - 3ms/step


[0.07063713669776917, 0.9803000092506409]

In [19]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               401920    
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535818 (2.04 MB)
Trainable params: 535818 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [21]:
model = keras.Sequential()
model.add(keras.Input(shape=(784)))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(256, activation="relu", name="my_layer"))
model.add(layers.Dense(10))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 512)               401920    
                                                                 
 my_layer (Dense)            (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535818 (2.04 MB)
Trainable params: 535818 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model = keras.Model(inputs=model.inputs, outputs=[model.layers[-1].output] )
model = keras.Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers] )

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)

In [ ]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

In [ ]:
print(model.summary())

# Functional API (A bit more flexible)

In [20]:
inputs = keras.Input(shape=(784))
x= layers.Dense(512, activation='relu', name="first_Layer")(inputs)
x= layers.Dense(256, activation='relu', name = "Second_Layer")(x)
outputs = layers.Dense(10, activation='softmax')(x)
model= keras.Model(inputs = inputs, outputs = outputs)


# Suggestions
- Try and see what accuracy you can get by increasing the modelm training for longer, .... etc you should get over 98.2% on the test set
- try other Optimizers than Adam for example Gradient Descent with Momentum, Adagrad and RMSprop
- if there is any difference if you remove the normalization of data?

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    # optimizer=keras.optimizers.Adam(learning_rate=0.001),
    # optimizer=keras.optimizers.SGD(learning_rate=0.001, momentum=0),
    # optimizer=keras.optimizers.Adagrad(learning_rate=0.001),
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
    metrics=["accuracy"],
)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training when the loss falls below 0.4

    Args:
      epoch (integer) - index of epoch (required but unused in the function definition below)
      logs (dict) - metric results from the training epoch
    '''

    # Check the loss
    # if(logs.get('loss') < 0.18):
    if(logs.get('accuracy') == 98.2 ):

      # Stop if threshold is met
      print("\nLoss is lower than 0.18 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=5, callbacks= [callbacks]) #verbose=2,

Epoch 1/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.4079 - accuracy: 0.8791
Epoch 2/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.3145 - accuracy: 0.9072
Epoch 3/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.2476 - accuracy: 0.9261
Epoch 4/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1979 - accuracy: 0.9413
Epoch 5/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.1627 - accuracy: 0.9516


In [ ]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)
# 313/313 - 1s - loss: 0.1249 - accuracy: 0.9607 - 1s/epoch - 3ms/step
# [0.12491389364004135, 0.9606999754905701]

313/313 - 1s - loss: 0.1588 - accuracy: 0.9541 - 1s/epoch - 3ms/step


[0.1587762087583542, 0.9541000127792358]

In [ ]:
print(model.summary())

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 784)]             0         
                                                                 
 first_Layer (Dense)         (None, 512)               401920    
                                                                 
 Second_Layer (Dense)        (None, 256)               131328    
                                                                 
 dense_20 (Dense)            (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None
